**Bảng phân công công việc:**

Thành viên | MSSV | Bài tập
--- | --- | ---
Đinh Anh Huy | 18110103 | 8.11, 8.23, 8.28
Nguyễn Đức Vũ Duy | 18110004 | 8.22, 8.26, 8.29

In [1]:
import numpy as np
import pandas as pd
import scipy.stats as stats
from matplotlib import pyplot as plt
import pandas as pd
import statsmodels.formula.api as smf
import statsmodels.api as sm
from sklearn.preprocessing import StandardScaler
from sklearn.decomposition import PCA
import warnings
warnings.filterwarnings("ignore")

###8.22

###Xét dữ liệu về bò tót ở bảng 1.10. Tận dụng 7 biến YrHgt, FtFrBody, PrctFFB, Frame, BkFat, SaleHi and SaleWt, thực hiện phân tích thành phần chính sử dụng ma trận hiệp phương sai $\boldsymbol{S}$ và ma trận tương quan $\boldsymbol{R}$. Phân tích cần bao gồm:


###(a) Xác định số lượng thành phần chính phù hợp để tóm tắt hiệu quả sự thay đổi ở mẫu. Vẽ scree plot để hỗ trợ xác định của bạn.

In [2]:
path='/content/T1-10.dat.txt'
df=pd.DataFrame(np.loadtxt(path)).iloc[:,2:]
df.columns=['YrHgt','FtFrBody','PrctFFB','Frame','BkFat','SaleHt','SaleWt']
df.head()

OSError: /content/T1-10.dat.txt not found.

In [ ]:
pd.DataFrame(np.loadtxt(path)).iloc[:,0]

In [ ]:
S=np.cov(df.T)
S

In [ ]:
R=np.corrcoef(df.T)
R

In [ ]:
def eigen_pairs(model, data):
    eigenvals = model.explained_variance_
    eigenvecs = model.components_

    print(">> Eigenanalysis")
    eigenvals_ = pd.DataFrame(eigenvals.reshape(1,-1))
    eigenvals_.columns = ['PC%s' % i for i in range(1, model.n_components_+1)]
    eigenvals_.index = ['Eigenvalue']
    
    eigenvecs_ = pd.DataFrame(eigenvecs.T)
    eigenvecs_.columns = ['PC%s' % i for i in range(1, model.n_components_+1)]
    eigenvecs_.index = data.columns
    print(eigenvals_)
    print("\n",eigenvecs_)
    return eigenvals_, eigenvecs_

In [ ]:
pca = PCA(n_components=7).fit(df)
eigenvals_, eigenvecs_=eigen_pairs(pca, df)

In [ ]:
def PCA_(matrix, n_components=None, correlation=False, column_names=None, return_=False):
    # Eigendecomposition
    eigenvals, eigenvecs = np.linalg.eigh(matrix)

    # Sort paires (eigenvalue, eigenvector) in descending order
    eigenpairs = [(abs(eigenval), eigenvec) for (eigenval, eigenvec) in zip(eigenvals, eigenvecs.T)]
    eigenpairs = sorted(eigenpairs, key = lambda pair: pair[0], reverse = True)

    # Select a subset from the rearranged Eigenvalues vector
    # If n_components is None, select all. Otherwise, select first n_components elements
    if n_components is None:
        eigenpairs_ = eigenpairs
    else:
        eigenpairs_ = eigenpairs[:n_components]

    # Construct weight matrix from above subset
    components_ = [eigenvec for (_, eigenvec) in eigenpairs_]
    explained_variance_ = [eigenval for (eigenval, _) in eigenpairs_]

    # Calculate percentage of cariance explained by each of the selected components
    explained_variance_ratio_ = []
    if correlation:
        for i in range(len(explained_variance_)):
            explained_variance_ratio_.append(explained_variance_[i]/len(explained_variance_))
    else:
        for i in range(len(explained_variance_)):
            explained_variance_ratio_.append(explained_variance_[i]/np.sum(eigenvals))
    explained_variance_ratio_ = np.array(explained_variance_ratio_)
        
    if correlation:
        print(">> Eigenanalysis of the Correlation Matrix")
    else:
        print(">> Eigenanalysis of the Covariance Matrix")
    
    eigenanalys = np.concatenate([np.array(explained_variance_).reshape(-1,1), explained_variance_ratio_.reshape(-1,1), np.cumsum(explained_variance_ratio_).reshape(-1,1)], axis=1)  
    eigenanalys = pd.DataFrame(eigenanalys.T)
    eigenanalys.columns = ['PC%s' % i for i in range(1, len(explained_variance_)+1)]
    eigenanalys.index = ['Eigenvalue', 'Proportion', 'Cumulative']
    
    eigenvecs_ = pd.DataFrame(np.array(components_).T)
    eigenvecs_.columns = ['PC%s' % i for i in range(1, len(explained_variance_)+1)]
    if column_names is not None:
        eigenvecs_.index = column_names
    print(eigenanalys)
    print("\n",eigenvecs_)
    
    if return_:
        return explained_variance_, components_, explained_variance_ratio_

In [ ]:
eigenvals, eigenvecs, proportions = PCA_(S, correlation=False, column_names=df.columns, return_=True)

2 thành phần mẫu chính đầu tiên là:
\begin{equation*}
\hat{y_1}=-0.0059x_1-0.487x_2-0.009x_3-0.0031x_4-0.000069x_5-0.0093x_6-0.8733x_7
\end{equation*}
\begin{equation*}
\hat{y_2}=0.00968x_1+0.873x_2+0.029196x_3+0.00489x_4-0.0005x_5+0.0086x_6-0.4872x_7
\end{equation*}

In [ ]:
fig, ax = plt.subplots(figsize=(10,5))

PC_values = np.arange(len(eigenvals)) + 1
ax.plot(PC_values, eigenvals, 'ro-', linewidth=2)
ax.set_title('Scree Plot')
ax.set_xlabel('Principal Component')
ax.set_ylabel('Eigenvalue')

plt.show()

Theo scree plot ở trên thì số lượng thành phần phù hợp để thực hiện PCA sẽ là 2. 

###(b) Tính thành phần chính của mẫu.

In [ ]:
from matplotlib.patches import Ellipse, Rectangle
def get_cov_ellipse(cov, centre, nstd, eig = False, **kwargs):
    """
    Return a matplotlib Ellipse patch representing the covariance matrix
    cov centred at centre and scaled by the factor nstd.

    """

    # Find and sort eigenvalues and eigenvectors into descending order
    eigvals, eigvecs = np.linalg.eigh(cov)
    order = eigvals.argsort()[::-1]
    eigvals, eigvecs = eigvals[order], eigvecs[:, order]
    # The anti-clockwise angle to rotate our ellipse by 
    vx, vy = eigvecs[:,0][0], eigvecs[:,0][1]
    theta = np.arctan2(vy, vx)

    # Width and height of ellipse to draw
    width, height = 2 * nstd * np.sqrt(eigvals)
    if eig:
        return Ellipse(xy=centre, width=width, height=height,
                   angle=np.degrees(theta), **kwargs), eigvals, eigvecs
    else: 
        return Ellipse(xy=centre, width=width, height=height,
                   angle=np.degrees(theta), **kwargs)

In [ ]:
pca = PCA(n_components=2).fit(df)
X_pca=pca.transform(df)
X_mean_1,X_mean_2=X_pca[:,0].mean(), X_pca[:,1].mean()
fig, ax = plt.subplots()
e = Ellipse(xy=(X_mean_1,X_mean_2),width=2*np.sqrt(eigenvals[0]),height=2*np.sqrt(eigenvals[1]),angle=np.arctan2(eigenvecs[0][0],eigenvecs[0][1]), edgecolor='blue', facecolor='None')
ax.scatter(x=X_pca[:,0],y=X_pca[:,1], c='red', s=25)
ax.scatter(X_mean_1,X_mean_2, c='green',s=25)
ax.add_patch(e)

###(d) Sử dụng những giá trị cho 2 thành phần chính đầu tiên, plot dữ liệu lên không gian 2 chiều với $\hat{y_1}$ là trục dọc và $\hat{y_2}$ trục ngang. Bạn có thể phân biệt những nhóm thể hiện 3 giống cattle ? có điểm ngoại lai không ?

In [ ]:
breed= pd.DataFrame(np.loadtxt(path)).iloc[:,0].values
df_new=pd.DataFrame(np.concatenate((X_pca,breed.reshape(-1,1)),axis=1))
df_new

In [ ]:
fig, ax = plt.subplots()
ax.scatter(df_new.iloc[:,1],df_new.iloc[:,0],c=df_new.iloc[:,2])
ax.set_title('Scatter plot')
ax.set_xlabel('$\hat{y_2}$')
ax.set_ylabel('$\hat{y_1}$')

Ta có thể thấy là các nhóm cho 3 chủng loại không tách biệt nhau ra nên chúng ta phân biệt các nhóm này. Ngoài ra chúng ta cũng nhận ra có 2 điểm outlier ở góc trên bên phải.

In [ ]:
import pylab
import statsmodels.api as sm
sm.qqplot(df_new.iloc[:,0],line='r')
plt.title('Q-Q plot using the first principal component')
plt.show()

Từ Q-Q plot, ta thấy dữ liệu sử dụng thành phần chính đầu tiên có tính chuẩn và có 1 điểm outlier ở góc trên bên phải. 

###8.26

###Sử dụg dữ liệu về profile tâm lý học ở bảng 4.6. Sử dụng 5 biến Indep, Supp, Benev, Conform và Leader, thực hiện phân tích thành phần chính sử dụng ma trận hiệp phương sai $\boldsymbol{S}$ và ma trận tương quan $\boldsymbol{R}$. Phân tích cần bao gồm

###(a) Xác định số lượng thành phần chính phù hợp để tóm tắt hiệu quả sự thay đổi ở mẫu. Vẽ scree plot để hỗ trợ xác định của bạn.



In [ ]:
path='/content/T4-6.DAT.txt'
df=pd.DataFrame(np.loadtxt(path)).iloc[:,:-2]
df.columns=['Indep','Supp','Benev','Conform','Leader']
df.head()

In [ ]:
S=np.cov(df.T)
S

In [ ]:
R=np.corrcoef(df.T)
R

In [ ]:
eigenvals, eigenvecs, proportions = PCA_(S, correlation=False, column_names=df.columns, return_=True)

2 thành phần mẫu chính đầu tiên là:
\begin{equation*}
\hat{y_1}=-0.579x_1+0.0417x_2+0.524x_3+0.493x_4-0.38x_5
\end{equation*}
\begin{equation*}
\hat{y_2}=0.079x_1+0.612x_2+0.219x_3-0.572x_4-0.494x_5
\end{equation*}

In [ ]:
fig, ax = plt.subplots(figsize=(10,5))

PC_values = np.arange(len(eigenvals)) + 1
ax.plot(PC_values, eigenvals, 'ro-', linewidth=2)
ax.set_title('Scree Plot')
ax.set_xlabel('Principal Component')
ax.set_ylabel('Eigenvalue')

plt.show()

Theo scree plot ở trên thì số lượng thành phần phù hợp để thực hiện PCA sẽ là 3, chiếm $86\%$ variance tổng. 

###(b) Tính thành phần chính của mẫu.

In [ ]:
pca = PCA(n_components=3).fit(df)
X_pca=pca.transform(df)
X_mean_1,X_mean_2=X_pca[:,0].mean(), X_pca[:,1].mean()
fig, ax = plt.subplots()
e = Ellipse(xy=(X_mean_1,X_mean_2),width=2*np.sqrt(eigenvals[0]),height=2*np.sqrt(eigenvals[1]),angle=np.arctan2(eigenvecs[0][0],eigenvecs[0][1]), edgecolor='blue', facecolor='None')
ax.scatter(x=X_pca[:,0],y=X_pca[:,1], c='red', s=25)
ax.scatter(X_mean_1,X_mean_2, c='green',s=25)
ax.add_patch(e)

###(c) Sử dụng những giá trị cho 2 thành phần chính đầu tiên, plot dữ liệu lên không gian 2 chiều với $\hat{y_1}$ là trục dọc và $\hat{y_2}$ trục ngang. Bạn có thể phân biệt những nhóm thể hiện 2 mức socioeconomic  hoặc 2 giới tính? có điểm ngoại lai không ?

In [ ]:
gender_socio= pd.DataFrame(np.loadtxt(path)).iloc[:,-2:].values
df_new=pd.DataFrame(np.concatenate((X_pca,gender_socio),axis=1))
df_new

In [ ]:
fig, ax = plt.subplots()
ax.scatter(df_new.iloc[:,1],df_new.iloc[:,0],c=df_new.iloc[:,3])
ax.set_title('Scatter plot with gender')
ax.set_xlabel('$\hat{y_2}$')
ax.set_ylabel('$\hat{y_1}$')

Theo scatterplot trên thì ta không thể phân biệt được 2 nhóm theo giới tính. Cũng từ 2 scatterplot ta thấy có 1 điểm ngoại lai.

In [ ]:
fig, ax = plt.subplots()
ax.scatter(df_new.iloc[:,1],df_new.iloc[:,0],c=df_new.iloc[:,4])
ax.set_title('Scatter plot with socioeconomic levels')
ax.set_xlabel('$\hat{y_2}$')
ax.set_ylabel('$\hat{y_1}$')

Tương tự như scatterplot trên thì ta cũng không phân biệt được 2 nhóm thể hiện 2 socioeconomic levels.

###(d) Xây dựng khoảng tin cậy $95\%$ cho $\lambda_1$, variance của thành phần chính đầu tiên của tổng thể từ ma trận hiệp phương sai. 

In [ ]:
alpha=0.05
n=df.shape[0]
import scipy.stats
z=scipy.stats.norm.ppf(1-alpha/2)
lowerbound=eigenvals[0]/(1+z*np.sqrt(2/n))
upperbound=eigenvals[0]/(1-z*np.sqrt(2/n))
print('95% confidence interval for $\lambda_1$ is: [{}, {}]'.format(lowerbound,upperbound))

###8.29

###Theo bài 5.28, sử dụng ma trận hiệp phương sai $\boldsymbol{S}$ cho 30 cases đầu tiên của dữ liệu dây chuyền thân xe hơi, đạt được thành phần chính mẫu.

###(a) Xây dựng $95\%$ ellipse format chart sử dụng 2 thành phần chính đầu tiên $\hat{y_1}$ và $\hat{y_2}$. Xác định vị trí xe mà ngoài tầm kiểm soát

In [ ]:
path='/content/T5-14.dat.txt'
df=pd.DataFrame(np.loadtxt(path)).iloc[:30,:]
df.head()

In [ ]:
S=np.cov(df.T)
S

In [ ]:
eigenvals, eigenvecs, proportions = PCA_(S, correlation=False, column_names=df.columns, return_=True)

2 thành phần chính đầu tiên giải thích $68\%$ tổng variance. Số lượng thành phần chính phù hợp để thực hiện phân tích sẽ là 4, giải thích $94.6\%$ tổng variance.

In [ ]:
pca = PCA(n_components=4).fit(df)
X_pca=pca.transform(df)
fig, ax = plt.subplots()
centre=(0,0)
#the upper 95% percentile of chi square with degree of freedom of 2 is 5.99
e = Ellipse(centre,width=2*np.sqrt(5.99*eigenvals[0]),height=2*np.sqrt(5.99*eigenvals[1]), edgecolor='blue', facecolor='None')
ax.scatter(x=X_pca[:,0],y=X_pca[:,1], c='red', s=25)
ax.add_patch(e)

Theo $95\%$ ellipse format chart ta có 2 điểm nằm ngoài hình ellipse. 2 điểm này chính là các quan trắc ngoài tầm kiểm soát.

In [ ]:
print(np.argmin(X_pca[:,1]))
print(np.argmax(X_pca[:,1]))

Theo đó, quan trắc thứ 3 và quan trắc thứ 11 nằm ngoài tầm kiểm soát.

###(b) Xây dụng một chart kiểm soát thay thế, dựa trên tổng bình phương $d_{ij}^2$ để kiểm xoát sự thay đổi trong các quan trắc ban đầu được tóm tắt bởi 4 thành phần chính còn lại. Diễn giải chart này.

In [ ]:
pca = PCA(n_components=6).fit(df)
X_pca=pca.transform(df)
X_pca.shape

In [ ]:
X_pca=X_pca-X_pca.mean(axis=0)
X_pca[:,2:]=X_pca[:,2:]**2
X=X_pca[:,2:].copy()
d_j=X.sum(axis=1)
d_j.shape

In [ ]:
d_u=d_j.mean()
s_u=d_j.std(ddof=1)
print(d_u)
print(s_u)

In [ ]:
c=s_u/(2*d_u)
v=2*(d_u**2)/s_u
print(c)
print(v)
#Từ đây, ta chọn bậc tự do v = 1

In [ ]:
from scipy.stats import chi2
#Tính giới hạn kiểm soát trên
upper_control_limit=c*chi2.ppf(0.95,1)
upper_control_limit

In [ ]:
#Tính T^2
X_new=pd.DataFrame(X_pca)
X_new['T^2']=X_new[2]**2/eigenvals[2]+X_new[3]**2/eigenvals[3]+X_new[4]**2/eigenvals[4]+X_new[5]**2/eigenvals[5]
X_new

In [ ]:
fig, ax = plt.subplots()
ax.plot(X_new['T^2'])
ax.axhline(upper_control_limit,c='red')
ax.axhline(0,c='red')

Từ đây ta thấy có 3 quan trắc ngoài tầm kiểm soát là quan trắc thứ 14, 17, 19.